# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached langchain_community-0.0.29-py3-none-any.whl.metadata (8.3 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/61.6 kB ? eta -:--:--
   ------------------- -------------------- 30.7/61.6 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 61.6/61.6 kB 829.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/47.7 kB ? eta -:--:--
   ---------------------------------- ----- 41.0/47.7 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 47.7/47.7 kB 797.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.8 MB 3.4 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.8 MB 1.3 MB/s eta 0:00:02
   --- --------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instagram 0.1.0 requires crewai[tools]<0.28.0,>=0.27.0, but you have crewai 0.28.8 which is incompatible.
linkedin 0.1.0 requires crewai[tools]<0.28.0,>=0.27.0, but you have crewai 0.28.8 which is incompatible.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [8]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [10]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [11]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

# John Doe

**Email:** john.doe@example.com  
**Phone:** (123) 456-7890  
**LinkedIn:** [linkedin.com/in/johndoe](https://linkedin.com/in/johndoe)  
**GitHub:** [github.com/johndoe](https://github.com/johndoe)  

---

## Professional Summary

Highly motivated and results-driven software engineer with 5+ years of experience in developing robust code for high-volume businesses. Skilled in problem-solving and team collaboration, with a proven ability to adapt to new technologies and tools. Seeking a challenging position to leverage my expertise in software development and contribute to the success of a dynamic company.

---

## Experience

### Software Engineer  
**Tech Solutions Inc.**  
*June 2019 – Present*  
- Developed and maintained web applications using JavaScript, React, and Node.js.
- Collaborated with cross-functional teams to design and implement new features.
- Optimized application performance, resulting in a 20% increase in speed.
- Mentored junior developers and conducted code reviews to ensure best practices.
- Utilized Agile methodologies to manage project timelines and deliverables.

### Junior Software Developer  
**Innovatech Corp.**  
*January 2017 – May 2019*  
- Assisted in the development of internal tools and client-facing applications.
- Participated in the full software development lifecycle, from requirements gathering to deployment.
- Wrote clean, maintainable code and implemented unit tests to ensure code quality.
- Resolved bugs and performed troubleshooting to maintain system functionality.
- Worked closely with senior developers to improve technical skills and knowledge.

---

## Education

### Bachelor of Science in Computer Science  
**State University**  
*September 2012 – May 2016*  
- Graduated with Honors
- Relevant coursework: Data Structures, Algorithms, Database Systems, Web Development

---

## Skills

- **Languages:** JavaScript, Python, Java, C++
- **Frameworks:** React, Node.js, Express, Django
- **Databases:** MySQL, PostgreSQL, MongoDB
- **Tools:** Git, Docker, Jenkins, AWS
- **Methodologies:** Agile, Scrum, Test-Driven Development (TDD)

---

## Projects

### Personal Portfolio Website  
- Designed and developed a personal portfolio website to showcase projects and skills.
- Implemented responsive design using HTML, CSS, and JavaScript.
- Deployed the website using AWS and maintained it for ongoing updates.

### E-commerce Platform  
- Collaborated with a team to build an e-commerce platform using React and Node.js.
- Integrated payment processing and user authentication features.
- Ensured high performance and scalability to handle a large number of users.

---

## Certifications

- Certified JavaScript Developer
- AWS Certified Solutions Architect – Associate

---

## Interests

- Open-source contribution
- Tech blogging
- Hiking and outdoor activities


## Creating Agents

In [12]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [13]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [14]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [15]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [16]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [17]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [18]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [19]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [20]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [21]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [22]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

- Dislplay the generated `tailored_resume.md` file.

In [23]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Smith

**Email:** noah.smith@example.com  
**Phone:** (555) 123-4567  
**LinkedIn:** [linkedin.com/in/noahsmith](https://linkedin.com/in/noahsmith)  
**GitHub:** [github.com/noahsmith](https://github.com/noahsmith)  

---

## Professional Summary

Accomplished Software Engineering Leader with 18 years of experience, specializing in managing remote and in-office teams. Expertise in multiple programming languages and frameworks, with a strong background in AI and data science. Successfully led major tech initiatives and startups, showcasing the ability to drive innovation and growth in the tech industry.

---

## Experience

### Software Engineering Leader  
**Tech Innovations LLC**  
*January 2004 – Present*  
- Led remote and in-office teams in developing desktop and mobile applications using common stacks.
- Managed full software development lifecycle, from requirements gathering to deployment.
- Implemented AI/ML models and utilized big data tools such as Hadoop, Spark, and Kafka.
- Collaborated with engineering teams and business stakeholders to drive technical discussions and innovation.
- Demonstrated exceptional project management and organizational skills, ensuring project success and growth.

### Software Engineer  
**Startup Technologies Inc.**  
*September 1999 – December 2003*  
- Developed consumer applications with a focus on data handling and privacy regulations.
- Utilized multiple front-end and object-oriented languages to create robust and scalable applications.
- Implemented UI/UX design principles to enhance user experience and engagement.
- Worked on various projects, including crewAI, machinery, active_model_serializers, gioco, sigma, and keeper.
- Engaged in code reviews and contributed to open-source projects to stay updated with industry best practices.

---

## Education

### Master of Business Administration (MBA)  
**University of Technology**  
*September 1997 – June 1999*  

### Bachelor of Science in Computer Science  
**State University**  
*September 1993 – May 1997*  
- Relevant coursework: Data Structures, Algorithms, Database Systems, Web Development

---

## Skills

- **Languages:** Java, Python, JavaScript, HTML/CSS
- **Frameworks:** Angular, React, Node.js
- **Databases:** MySQL, MongoDB
- **Tools:** Git, Docker, Jenkins, Apache
- **Methodologies:** Agile, Scrum, Test-Driven Development (TDD)

---

## Projects

### crewAI  
- Collaborated on the development of crewAI, an AI-powered application for team management.
- Implemented AI/ML models to optimize team performance and collaboration.
- Contributed to the project's success by utilizing big data tools for data analysis and insights.

### machinery  
- Led the development of machinery, a mobile application for remote equipment monitoring.
- Utilized React and Node.js to create a responsive and scalable application.
- Integrated real-time data processing for efficient equipment maintenance and monitoring.

---

## Certifications

- AWS Certified Solutions Architect – Associate
- Certified JavaScript Developer

---

## Interests

- Open-source contribution
- Tech leadership
- Data science and AI advancements

- Dislplay the generated `interview_materials.md` file.

In [24]:
display(Markdown("./interview_materials.md"))

## Interview Questions and Talking Points for Noah Smith

### Full Stack Development Experience
1. How have you demonstrated your experience as a Full Stack Developer in previous roles?
2. Can you discuss a project where you developed both desktop and mobile applications using common stacks?
3. What front-end languages and libraries are you most comfortable working with, and can you provide examples of projects where you utilized them?

### Programming Languages and Frameworks
1. Describe your knowledge and experience with object-oriented languages like Java and Python.
2. Have you worked with frameworks such as Angular, React, and Node.js? If so, can you share a project where you applied these frameworks effectively?

### Database and Web Servers
1. What is your familiarity with databases like MySQL and MongoDB, as well as web servers like Apache?
2. Can you explain a project where you integrated databases and web servers to enhance the overall user experience?

### Communication Skills and Technical Discussions
1. How do you approach in-depth technical discussions with both the engineering team and business stakeholders?
2. Share an example of a project where your communication skills played a crucial role in driving technical innovation and collaboration.

### Project Management and Organizational Skills
1. How do you ensure project success and growth through your project management and organizational skills?
2. Discuss a challenging project where your strong project management skills were key to the successful outcome.

### Education and Certifications
1. How has your educational background in Computer Science and an MBA contributed to your success as a Software Engineering Leader?
2. How do you stay updated with current trends and advancements in the industry, especially in AI/ML models and big data tools?

### Personal Projects and Interests
1. Can you share insights from your personal projects like crewAI and machinery, highlighting your diverse skills and contributions to the tech community?
2. How do you balance open-source contributions, tech leadership interests, and data science advancements in your career development?

These interview questions and talking points will help highlight Noah Smith's key experiences, skills, and qualifications in relation to the job requirements for the Full Stack Developer position.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)